In [1]:
suppressPackageStartupMessages(
    {suppressWarnings({
        library(tidyverse)
        library(repr)
        library(tidymodels)
        library(tidyr)
        library(ggplot2)        
        library(scales)
        library(patchwork) 
        library(purrr)
        library(dplyr)
        library(GGally)
        library(ISLR)
    })
})

In [2]:
# Size and general style set up
options(repr.plot.width = 6, repr.plot.height = 4, repr.matrix.max.rows = 7,readr.show_col_types = FALSE)

# Load Data
player_data <- read_csv("https://raw.githubusercontent.com/FabianoGLentini/player-subscription-ml/refs/heads/main/data/players.csv")


# Data Science Project: Project Final Report


## Introduction: 
For this project, we are analyzing data collected by researchers in Computer Science at UBC. They have set up a MineCraft server that allows them to record consenting players’ behaviour and characteristics for study. In particular, we want to answer the question: **What players' `Age`, `gender`, and `played_hours` can best predict whether they would `subscribe` to a gaming newsletter, and how does it differ from players `experience`?** In this project, we will focus on the `players.csv` dataset, as it is of most use to us to answer our question. The `players.csv` set contains 196 observations and 7 variables. This data is already in its tidy form because every column is a single variable, every row is a single observation, and every cell is a single value.  We can see that there are three types of variables here: character, logical, and double. 

**Character Variables:**

`hashedEmail` (email of player that has been converted into a unique string of characters, for privacy)

`name` (first name of player)

`gender` (gender of player)

`experience` (skill level of player)

**Double Variables:**

`played_hours` (number of hours spent on the game by each player)

`Age` (age in years of each player)

**Logical variable:**

`subscribe` (whether or not the player is subscribed to the game newsletter) 

For our question, we want to know if a player's age, gender, and amount of played hours can determine if they will subscribe to the game newsletter or not. We chose these three predictor variables because they are all characteristics of each individual player that can allow us to group them and determine which type of player is most likely to subscribe. Additionally, we want to determine if any difference in experience across players will be a factor in whether or not a player is subscribed.

A potential issue with this data under the `played_hours` variable, we can see that many players have 0 hours played. This could cause weird results when we begin to wrangle. Furthermore, some values under the `gender` variable contain very little sample sizes, making it hard to scale these categories. 

## Methods & Results:

### Set up and intro to data: "TODO should rename later." 

In [3]:
# Wrangle data
#TODO will need additional wranggling due to poor or few varied sample categories
player_df <- player_data |> # TODO must consider reworking a joining of some labels in gender etc due to low representaions
            select( subscribe, gender, played_hours, experience, Age) |>
            drop_na() |>
            mutate(
                subscribe = as_factor(subscribe),  # lgl -> fct for analysis and modeling
                gender = as_factor(gender),        # chr -> fct for analysis and modeling
                played = factor(as.logical(played_hours)), # TODO may not use 
                experience = as_factor(experience) # chr -> fct for analysis and modeling
            ) |>
             mutate(gender = fct_recode(gender, # Aggregate some of the gender to balance out count and avoid missclassification due to underepresented categories
                                    "Non-binary/Other" = "Agender",
                                    "Non-binary/Other" = "Non-binary",
                                     "Non-binary/Other" = "Two-Spirited",
                                    "Non-binary/Other" = "Other"
                                      ))
 # Removed row with NA values, as it may distort the model      

# TODO FABIO check if player_hours should adjust to use a binary outcome, 
# either played or didn't play at all, or if any of the predictor should be removed.


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `gender = fct_recode(...)`.
Caused by warning:
! Unknown levels in `f`: Other”


### Training and Analysis:

In [4]:
# head(player_df) # TODO DELETE tmp  for set up 

In [5]:
# SPLIT/SCALE DATA
set.seed(2025) # Don't change
# Prep for modelling
ply_df <- select(player_df, Age, subscribe, gender, played_hours, experience)  # Exclude row_id & experience for modeling purposes

# Split step
player_split <- initial_split(ply_df, prop = 0.70, strata = subscribe) 
player_train <- training(player_split)
player_test <- testing(player_split)

In [6]:
#TODO DELETE 
## tr_sub_counts <- ggplot(player_train, aes(x = subscribe, fill = subscribe)) +
#                 geom_bar() + 
#                 ylim(c(0,100)) + 
#                 guides(fill = "none") +
#                 labs(x = "Subscribe status", y = "Count of players") + 
#                 ggtitle("Training set")
                
# tst_sub_counts <- tr_sub_counts %+% player_test +
#                 ggtitle("Testing Set")


                

# tr_sub_counts + tst_sub_counts

player_split

<Training/Testing/Total>
<135/59/194>

**Data Split:**

Due to a small, fairly imbalanced data set of 194 data points, we will use an 'initial_split' of 70% (135) training and 30% (59) testing sets to try to balance the split across all our predictors. Our goal is to train a model to predict the outcome of subscription, thus our strata will be set to `subscribe` variable. 


In [7]:
# Recipes:
# Scale/Recipe

# Note: A = Age, G = gender, H = played_hours and E = experience
# Recipe 01:
# Age + gender + played_hours
rc_AGH <- recipe(subscribe ~ Age + gender + played_hours, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours, Age) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())
# # Recipe 02:
# Age + played_hours
rc_AH <- recipe(subscribe ~ Age + played_hours, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours, Age) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 03:
# Age + gender 
rc_AG <- recipe(subscribe ~ Age + gender, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(Age) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 04:
# gender + played_hours
rc_GH <- recipe(subscribe ~ gender + played_hours, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 05:
# Age
rc_A <- recipe(subscribe ~ Age, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(Age) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 06:
# gender
rc_G <- recipe(subscribe ~ gender, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 07:
# played_hours
rc_H <- recipe(subscribe ~ played_hours, player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())
# Recipe 08:
# Age + gender + played_hours + experience
rc_AGHE <- recipe(subscribe ~ Age + gender + played_hours + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours, Age) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 09:
# gender + played_hours + experience
rc_GHE <- recipe(subscribe ~ gender + played_hours + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_log(played_hours) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 10:
# played_hours + experience
rc_HE <- recipe(subscribe ~played_hours + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 11:
# gender + played_hours + experience
rc_E <- recipe(subscribe ~ experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 12:
# gender + experience
rc_GE <- recipe(subscribe ~ gender + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 13:
# Age + played_hours + experience
rc_AHE <- recipe(subscribe ~ Age + played_hours + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 14:
# Age + experience
rc_AE <- recipe(subscribe ~ Age+ experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())

# Recipe 15:
# Age + gender + experience
rc_AGE <- recipe(subscribe ~ Age + gender + experience, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors()) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_predictors())



**Recipes:**

We've put together 15 recipes to explore all the different combinations of predictors that we will train and test our model on to see which combination performs best at predicting `subscribe` outcomes. We will train using our player training dataset. Our recipe must standardize our factor and numerical variables to remove discrepancies in the evaluation weight. We will be using step_dummy to turn our factors/categorical variables (e.g. experience, gender) into dummy numerical variables, then use step_zv

> **Combinations of our recipes:**
>
> 1. Age + gender + played_hours + experience
> 2. Age + played_hours
> 3. Age + gender
> 4. Gender + played_hours
> 5. Gender + played_hours + experience
> 6. played_hours + experience
> 7. Gender + experience
> 8. Age + played_hours + experience
> 9. Age + experience
> 10. Age + gender + experience
> 11. Age
> 12. gender
> 13. played_hours
> 14. experience
> 15. Age + gender + experience

In [8]:
# Spec set up
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

#TODO FABIO ... search refractor options to reduce code clutter
# Check mean and standard error through collect_metrics


In [9]:
# TODO FABIO confirm that not having set.seed here is fine as long as set.seed above code cell goes first
# K-fold cross-validation
set.seed(1234) # Don't change
kfolds <- vfold_cv(player_train, v = 5, strata = subscribe)
k_vals = tibble(neighbors = seq(from = 1, to = 10, by = 1))


In [10]:
# TODO FABIO ~ write up spec use and impl of vfold..?

In [11]:
#TODO FABIO write reasonin + graph to show fold outcome
#Note the reason of using 10 10-fold is due to the small size data,
#hence it will improve the estimate and 

In [12]:
# Simplifying multi-model testing: https://www.youtube.com/watch?v=YZqbOATpjM4&t=139s
# Workflow Step
workflow_set <- workflow_set(
    preproc = list(
        AGH = rc_AGH,
        AH = rc_AH,
        AG = rc_AG,
        GH = rc_GH,
        A = rc_A,
        G = rc_G,
        H = rc_H,
        AGHE = rc_AGHE,
        GHE = rc_GHE,
        HE = rc_HE,
        E = rc_E,
        GE = rc_GE,
        AHE = rc_AHE,
        AE = rc_AE,
        AGE = rc_AGE
    ),
    models = list(knn_tune),
    cross = TRUE
)

# Tune workflow
# set.seed(22)

knn_tuned_set <- workflow_map(
    workflow_set,
    "tune_grid",
    resamples = kfolds,
    grid = k_vals,
    seed = 22
)


Execution stopped; returning current results



In [13]:
#TODO FABIO write workflow step use case/what it functionally is doing 

In [14]:

options(repr.plot.width = 15, repr.plot.height = 5)
# Get metrics all in one
all_metrics_collected <- workflow_map(
    knn_tuned_set,
    metrics = metric_set(accuracy, recall, precision)
) 

all_metrics_collected_res <- collect_metrics(all_metrics_collected)

→ A | warning: Column `played_hours` returned NaN, because variance cannot be calculated and
               scaling cannot be used. Consider avoiding `Inf` or `-Inf` values and/or setting
               `na_rm = TRUE` before normalizing.

There were issues with some computations   A: x1

→ B | error:   incorrect number of dimensions

There were issues with some computations   A: x1
There were issues with some computations   A: x5   B: x5



Warning message:
“All models failed. Run `show_notes(.Last.tune.result)` for more information.”
→ A | warning: Column `played_hours` returned NaN, because variance cannot be calculated and
               scaling cannot be used. Consider avoiding `Inf` or `-Inf` values and/or setting
               `na_rm = TRUE` before normalizing.

There were issues with some computations   A: x1

→ B | error:   incorrect number of dimensions

There were issues with some computations   A: x1
There were issues with some computations   A: x5   B: x5



Warning messag

ERROR: [1m[33mError[39m in `dplyr::mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `metrics = purrr::map(result, collect_metrics, summarize
  = summarize)`.
[1mCaused by error in `purrr::map()`:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `estimate_tune_results()`:[22m
[33m![39m All models failed. Run `show_notes(.Last.tune.result)` for more information.


In [ ]:
# res_All_met_tuned

In [ ]:
# all_metrics_collected_res
#TODO TMP trying to regain neighbours
res_All_met_tuned <- all_metrics_collected |>
            rowwise() |>
            mutate(metrics = list(collect_metrics(result)),
                  wflow_id = fct_recode(wflow_id,
                      "Age, gender and played hours" = "AGH_nearest_neighbor",
                      "Gender and played hours" = "GH_nearest_neighbor",
                      "Gender" = "G_nearest_neighbor",
                      "Age and gender" = "AG_nearest_neighbor",
                      "Age and played hours" = "AH_nearest_neighbor",
                      "Age" = "A_nearest_neighbor",
                      "Played hours" = "H_nearest_neighbor",
                    "Age, Gender, played_hours and experience" = "AGHE_nearest_neighbor",
                   "Gender, played_hours and experience" = "GHE_nearest_neighbor",   
                    "played_hours and experience" = "HE_nearest_neighbor", 
                     "experience" = "E_nearest_neighbor", 
                     "Gender and experience" = "GE_nearest_neighbor",
                     "Age, played_hours and experience" = "AHE_nearest_neighbor",
                     "Age and experience" = "AGE_nearest_neighbor",
                      "Age, Gender and experience" = "AGE_nearest_neighbor",
                  )) |>
            unnest(metrics) |>
            select(wflow_id, .metric, mean, neighbors, std_err) |>
            arrange(desc(mean))
# ADD RANKING
ranked_met <- res_All_met_tuned |>
            mutate(rank = seq(1, nrow(res_All_met_tuned), 1))
# ranked_met
# Visualize result
options(repr.plot.width = 8, repr.plot.height = 5)

plot_met_tmp <- ggplot(ranked_met, aes(x = rank, y = mean, color = wflow_id)) +
                geom_point() +
                 # geom_errorbar(aes(ymin = mean - std_err, ymax = mean + std_err), width=.2, #TODO TMP test to see if use line but std_err may not be correct
                 # position = position_dodge(0.05)) + 
                ylim(c(0,1)) + 
                labs(x = "Ranke by metric mean", y = "Metric mean percentage", color = "Predictor Combinations") 

# plot_met_tmp


In [ ]:
# METRICS set up Graphs
# Accuracy
acc_met <- res_All_met_tuned |>
            filter(.metric ==  "accuracy")
acc_met_rank <- mutate(acc_met, rank = seq(1, nrow(acc_met), 1)) |>
                slice(1:50)
acc_met_plot <- plot_met_tmp %+% acc_met_rank +
                guides(color = "none") + 
                ggtitle("Ranked by Accuracy Mean") 

                


# Precision
prec_met <- res_All_met_tuned |>
            filter(.metric ==  "precision")
prec_met_rank <- mutate(prec_met, rank = seq(1, nrow(prec_met), 1)) |>
                slice(1:50)
prec_met_plot <- plot_met_tmp %+% prec_met_rank +
                guides(color = "none") + 
                ggtitle("Ranked by Precision Mean") 
                



# Recall
rec_met <- res_All_met_tuned |>
            filter(.metric ==  "recall")
rec_met_rank <- mutate(rec_met, rank = seq(1, nrow(rec_met), 1)) |>
               slice(1:50)
rec_met_plot <- plot_met_tmp %+% rec_met_rank +
                ggtitle("Ranked by Recall Mean")


In [ ]:
#TODO conf amat graph of each category ???


In [ ]:
# All results side by side
options(repr.plot.width = 15, repr.plot.height = 5)
#TODO fabio add std_err line but also combine color and shape legend
# side_by_side_met
acc_met_plot + prec_met_plot + rec_met_plot

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# Accuracy tuned metric plot
acc_met_plot

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# Accuracy tuned metric prec
prec_met_plot

In [ ]:

#TODO write about the result prec

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# Accuracy tuned metric rec
rec_met_plot

In [ ]:
#TODO write about the result rec

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# LINE Graph showing k performance for each recipe
overlapped_line_acc <- ggplot(acc_met, aes(x = neighbors, y = mean, color = wflow_id)) +
                        geom_line() +
                        geom_point() + 
                        ylim(c(0, 1)) + 
                        scale_x_continuous(breaks = seq(0,5,1)) +
                        labs(
                            x = "K neighbors", 
                            y = "Accuracy percentage",  
                            color = "Predictor Combinations",
                        )  +
                        ggtitle("K-nn performance of K by Accuracy")
                        

overlapped_line_acc

In [ ]:
#TODO FABIO discuss on concerns for high k doing so well, should concider reducing k for some options maybe?
# Accuracy

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# LINE Graph showing k performance for each recipe Using precision
overlapped_line_prec <- overlapped_line_acc %+% prec_met +
                        ggtitle("K-nn performance of K by Precision")
overlapped_line_prec

In [ ]:
#TODO FABIO discuss on concerns for high k doing so well, should concider reducing k for some options maybe?
# Precission

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
# LINE Graph showing k performance for each recipe Using precision
overlapped_line_rec <- overlapped_line_acc %+% rec_met +
                        ggtitle("K-nn performance of K by Recall")
overlapped_line_rec

In [ ]:
#TODO FABIO discuss on concerns for high k doing so well, should concider reducing k for some options maybe?
# Recall

In [ ]:
# SELECT FINAL MODEL AND TRAIN 
set.seed(22)
#TMP DELETE
# player_test
# player_train

# Model Age + played_hours with k = 5 recipe(rc_AH)
final_md_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
            set_engine("kknn") |>
            set_mode("classification")

final_md_fit <- workflow() |>
            add_recipe(rc_AE) |>
            add_model(final_md_spec) |>
            fit(player_train)
# final_md_fit


########
final_md_pred <- predict(final_md_fit, player_test) |>
                bind_cols(player_test)
# final_md_pred
final_md_met <- final_md_pred |>
                metrics(truth = subscribe, estimate = .pred_class)
final_md_met
final_md_mat <- conf_mat(final_md_pred, truth = subscribe, estimate = .pred_class)
final_md_mat

In [ ]:
# 

### Player Type Exploration: "TODO may need renaming"

## Discussion:

### Reference 

##### Data Science programming techniques and approaches:
- **workflowset:** https://workflowsets.tidymodels.org/
- **Clustering prediction:** https://www.youtube.com/watch?v=z57i2GVcdww
- **Simplifying multi-model set up + testing:** https://www.youtube.com/watch?v=YZqbOATpjM4
- **Tuning and comparing models using Workflowse:** https://workflowsets.tidymodels.org/articles/tuning-and-comparing-models.html
- **Useful example of report/tutorial for tidy models:** https://optimumsportsperformance.com/blog/k-nearest-neighbor-tidymodels-tutorial/
- **Additional breakdown on workflow_set usage:** https://www.youtube.com/watch?v=R95lWUDtL5A
- **Workflow_set documentation on results:** https://workflowsets.tidymodels.org/reference/collect_metrics.workflow_set
- **Workflow_set tunning and comparing documentation:** https://workflowsets.tidymodels.org/articles/tuning-and-comparing-models
- **Plot ggpair examples breakdown for reference:** https://r-charts.com/correlation/ggpairs/
- **margin of error plot template models:** https://www.sthda.com/english/wiki/ggplot2-error-bars-quick-start-guide-r-software-and-data-visualization

In [ ]:
#TODO tmp info grounds

?guides